In [1]:
from pLaplace_1D import test_runner_1D

# all possible types of inputs (not all combinations allowed)
_ = ["numpy", "numba", "jax", "none"]  # "val_grad"
_ = ["Laplace", "SFD", "dense", "sparse"]  # "hess"
_ = ["my_newton", "my_trust_region",
     "scipy_trust-exact", "scipy_Newton-CG", "scipy_BFGS", "scipy_L-BFGS-B"]  # "minimizer"
_ = ["zero", "laplace"]  # "initial_guess"

problem_setting = {"p": 3,   # p in pLaplace
                   "a": -1,  # left bound of computational domain
                   "b": 1,   # right bound of computational domain
                   "f": "default"}  # source term, constant 10

sizes = [10, 100, 1000]


**-16.865480854231357**

# Conclusions first
- fastest value and grad evaluation have numba (rewrite of matlab code)
- fastest method without the use of user provided hessian is `L-BFGS-B` (scipy)
- method similar to the "best" in the matlab code is `Newton-CG` (scipy) with approximation of hessian using numrical differentiation SFD (rewrite of matlab code)
- fastest method overall is my implementation of `newton` with linesearch using golden section with Laplace as a constant approximation of the hessian 

In [2]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "none",
                    "minimizer": ["scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('numba', 'none', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'scipy_Newton-CG_hessp', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                       $n=10$        \
Option                                             time prep/sol iters   
('numba', 'none', 'scipy_L-BFGS-B', 'zero')        1.431 / 0.003  16.0   
('numba', 'SFD', 'scipy_Newton-CG_hessp', 'zero')  0.679 / 0.012  12.0   
('numba', 'Laplace', 'my_newton', 'zero')          0.596 / 0.002  15.0   

$\mathcal{N}_{dof}$                                      $n=100$         \
Option                                             time prep/sol  iters   
('numba', 'none', 'scipy_L-BFGS-B', 'zero')        0.733 / 0.013  300.0   
('numba', 'SFD', 'scipy_Newton-CG_hessp', 'zero')  0.927 / 0.027   17.0   
('numba', 'Laplace', 'my_newton', 'zero')          0.568 / 0.006   37.0   

$\mathcal{N}_{dof}$                                     $n=1000$          
Option                                             time prep/sol   iters  
('numba', 'none', 'scipy_L-BFGS-B', 'zero')        0.567 / 0.365  2853.0  
('numba', 'SFD', 'scipy_Newton-CG_hessp', 'zero')  0.616 / 0.658    18.0  
('numba', 'Laplace', 'my_newton', 'zero')          0.904 / 0.033    63.0

In [3]:
print(combined_df2.to_latex(index=False))


\begin{tabular}{rrrrrr}
\toprule
\$n=10\$ & \multicolumn{2}{l}{\$n=100\$} & \multicolumn{2}{l}{\$n=1000\$} \\
  time & iters &    time & iters &     time &  iters \\
\midrule
 0.003 &  16.0 &   0.013 & 300.0 &    0.365 & 2853.0 \\
 0.012 &  12.0 &   0.027 &  17.0 &    0.658 &   18.0 \\
 0.002 &  15.0 &   0.006 &  37.0 &    0.033 &   63.0 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_11173/1938860787.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(combined_df2.to_latex(index=False))


In [4]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('numba', 'SFD', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                        $n=10$        \
Option                                              time prep/sol iters   
('numba', 'SFD', 'my_newton', 'zero')               0.667 / 0.004   8.0   
('numba', 'SFD', 'my_trust_region', 'zero')         0.667 / 0.018   9.0   
('numba', 'Laplace', 'my_newton', 'zero')           0.684 / 0.001  15.0   
('numba', 'Laplace', 'my_trust_region', 'zero')      0.684 / 0.03  14.0   
('numba', 'SFD', 'my_newton', 'laplace')            0.752 / 0.005   4.0   
('numba', 'SFD', 'my_trust_region', 'laplace')      0.752 / 0.015   7.0   
('numba', 'Laplace', 'my_newton', 'laplace')        0.603 / 0.001   8.0   
('numba', 'Laplace', 'my_trust_region', 'laplace')  0.603 / 0.028  14.0   

$\mathcal{N}_{dof}$                                       $n=100$        \
Option                                              time prep/sol iters   
('numba', 'SFD', 'my_newton', 'zero')               0.848 / 0.008   8.0   
('numba', 'SFD', 'my_trust_region', 'zero')         0.848 / 0.055  18.0   
('numba', 'Laplace', 'my_newton', 'zero')           0.686 / 0.009  37.0   
('numba', 'Laplace', 'my_trust_region', 'zero')     0.686 / 0.203  43.0   
('numba', 'SFD', 'my_newton', 'laplace')            0.615 / 0.007   5.0   
('numba', 'SFD', 'my_trust_region', 'laplace')      0.615 / 0.023  10.0   
('numba', 'Laplace', 'my_newton', 'laplace')        0.606 / 0.009  35.0   
('numba', 'Laplace', 'my_trust_region', 'laplace')  0.606 / 0.086  30.0   

$\mathcal{N}_{dof}$                                      $n=1000$        
Option                                              time prep/sol iters  
('numba', 'SFD', 'my_newton', 'zero')               0.864 / 0.022  13.0  
('numba', 'SFD', 'my_trust_region', 'zero')         0.864 / 0.115  25.0  
('numba', 'Laplace', 'my_newton', 'zero')           0.822 / 0.033  63.0  
('numba', 'Laplace', 'my_trust_region', 'zero')     0.822 / 0.319  73.0  
('numba', 'SFD', 'my_newton', 'laplace')             0.826 / 0.01   8.0  
('numba', 'SFD', 'my_trust_region', 'laplace')      0.826 / 0.046  11.0  
('numba', 'Laplace', 'my_newton', 'laplace')        0.753 / 0.029  61.0  
('numba', 'Laplace', 'my_trust_region', 'laplace')  0.753 / 0.287  66.0

# Solvers with only grad information
## Comparison of implementations (numpy vs numba vs jax)

In [5]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "none",
                    "minimizer": ["scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "none",
                    "minimizer": ["scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "none",
                    "minimizer": ["scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Function value:


,10,100,1000
"('numpy', 'none', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'none', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'none', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                 $n=10$        \
Option                                       time prep/sol iters   
('numpy', 'none', 'scipy_L-BFGS-B', 'zero')  0.002 / 0.003  16.0   
('numba', 'none', 'scipy_L-BFGS-B', 'zero')  0.848 / 0.002  16.0   
('jax', 'none', 'scipy_L-BFGS-B', 'zero')    0.177 / 0.003  16.0   

$\mathcal{N}_{dof}$                                $n=100$         \
Option                                       time prep/sol  iters   
('numpy', 'none', 'scipy_L-BFGS-B', 'zero')    0.0 / 0.031  259.0   
('numba', 'none', 'scipy_L-BFGS-B', 'zero')  0.884 / 0.022  300.0   
('jax', 'none', 'scipy_L-BFGS-B', 'zero')    0.165 / 0.038  327.0   

$\mathcal{N}_{dof}$                               $n=1000$          
Option                                       time prep/sol   iters  
('numpy', 'none', 'scipy_L-BFGS-B', 'zero')  0.001 / 0.641  3416.0  
('numba', 'none', 'scipy_L-BFGS-B', 'zero')  0.916 / 0.363  2853.0  
('jax', 'none', 'scipy_L-BFGS-B', 'zero')    0.248 / 0.802  3534.0

## For numba a comparison of multiple minimizators

In [6]:
all_solvers = []
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["scipy_TNC", "scipy_BFGS", "scipy_CG", "scipy_L-BFGS-B"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('numba', 'SFD', 'scipy_TNC', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'scipy_BFGS', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'scipy_CG', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'scipy_L-BFGS-B', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                $n=10$             $n=100$  \
Option                                      time prep/sol iters time prep/sol   
('numba', 'SFD', 'scipy_TNC', 'zero')       0.689 / 0.001  13.0  0.79 / 0.022   
('numba', 'SFD', 'scipy_BFGS', 'zero')      0.689 / 0.005  22.0  0.79 / 0.041   
('numba', 'SFD', 'scipy_CG', 'zero')        0.689 / 0.006  39.0  0.79 / 0.102   
('numba', 'SFD', 'scipy_L-BFGS-B', 'zero')  0.689 / 0.001  16.0  0.79 / 0.015   

$\mathcal{N}_{dof}$                                       $n=1000$           
Option                                       iters   time prep/sol    iters  
('numba', 'SFD', 'scipy_TNC', 'zero')         57.0   0.832 / 0.614    558.0  
('numba', 'SFD', 'scipy_BFGS', 'zero')       119.0  0.832 / 84.006   1000.0  
('numba', 'SFD', 'scipy_CG', 'zero')        1022.0   0.832 / 5.808  31931.0  
('numba', 'SFD', 'scipy_L-BFGS-B', 'zero')   300.0   0.832 / 0.489   2853.0

# Solvers using hessians 
**in scipy**
- `trust-exact` needs dense matrix, no reasonable warkaround
- `newton-cg` needs hessian-vector product

**own implementation of trust region method**

## Comparison of multiple implementation of Hessian approximation
- exact from jax autodiff - both dense and sparse version
- approx using SDF (copy of matlab code)
- constant approximation using Laplace (p=2)

In [7]:
all_solvers = []
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_newton"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('jax', 'sparse', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('jax', 'SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                   $n=10$        \
Option                                         time prep/sol iters   
('jax', 'sparse', 'my_newton', 'laplace')      0.717 / 0.057   4.0   
('jax', 'SFD', 'my_trust_region', 'zero')      0.147 / 0.081   9.0   
('jax', 'Laplace', 'my_trust_region', 'zero')  0.138 / 0.096  14.0   

$\mathcal{N}_{dof}$                                  $n=100$        \
Option                                         time prep/sol iters   
('jax', 'sparse', 'my_newton', 'laplace')      0.819 / 0.032   5.0   
('jax', 'SFD', 'my_trust_region', 'zero')      0.168 / 0.161  18.0   
('jax', 'Laplace', 'my_trust_region', 'zero')  0.145 / 0.498  43.0   

$\mathcal{N}_{dof}$                                 $n=1000$        
Option                                         time prep/sol iters  
('jax', 'sparse', 'my_newton', 'laplace')       0.883 / 0.05   6.0  
('jax', 'SFD', 'my_trust_region', 'zero')      0.232 / 0.448  28.0  
('jax', 'Laplace', 'my_trust_region', 'zero')  0.166 / 0.926  73.0

In [9]:
all_solvers = []
all_solvers.append({"val_grad": "jax",
                    "hess": "dense",
                    "minimizer": ["scipy_trust-exact", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_trust_region", "scipy_Newton-CG_hessp"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('jax', 'dense', 'scipy_trust-exact', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'dense', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'sparse', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'SFD', 'scipy_Newton-CG_hessp', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                     $n=10$        \
Option                                           time prep/sol iters   
('jax', 'dense', 'scipy_trust-exact', 'zero')    0.397 / 0.025  10.0   
('jax', 'dense', 'my_trust_region', 'zero')      0.397 / 0.074   9.0   
('jax', 'sparse', 'my_trust_region', 'zero')     0.321 / 0.068   9.0   
('jax', 'SFD', 'my_trust_region', 'zero')        0.139 / 0.081   9.0   
('jax', 'SFD', 'scipy_Newton-CG_hessp', 'zero')  0.139 / 0.016  12.0   
('jax', 'Laplace', 'my_trust_region', 'zero')    0.137 / 0.094  14.0   

$\mathcal{N}_{dof}$                                    $n=100$        \
Option                                           time prep/sol iters   
('jax', 'dense', 'scipy_trust-exact', 'zero')      0.6 / 0.153  55.0   
('jax', 'dense', 'my_trust_region', 'zero')        0.6 / 0.543  48.0   
('jax', 'sparse', 'my_trust_region', 'zero')     0.353 / 0.426  47.0   
('jax', 'SFD', 'my_trust_region', 'zero')        0.154 / 0.166  18.0   
('jax', 'SFD', 'scipy_Newton-CG_hessp', 'zero')  0.154 / 0.047  19.0   
('jax', 'Laplace', 'my_trust_region', 'zero')    0.155 / 0.479  43.0   

$\mathcal{N}_{dof}$                                    $n=1000$         
Option                                            time prep/sol  iters  
('jax', 'dense', 'scipy_trust-exact', 'zero')    0.509 / 40.809  503.0  
('jax', 'dense', 'my_trust_region', 'zero')      0.509 / 29.012  489.0  
('jax', 'sparse', 'my_trust_region', 'zero')      0.419 / 7.749  485.0  
('jax', 'SFD', 'my_trust_region', 'zero')         0.205 / 0.405   28.0  
('jax', 'SFD', 'scipy_Newton-CG_hessp', 'zero')   0.205 / 0.687   19.0  
('jax', 'Laplace', 'my_trust_region', 'zero')     0.153 / 0.877   73.0

# Comparison of implementations (numpy vs numba vs jax)
 - for trust region
 - also for newton method as both SFD and Laplace converge

In [10]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numpy",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "zero",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRankWarning
MatrixRank

/usr/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


,10,100,1000
"('numpy', 'Laplace', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numpy', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('numpy', 'SFD', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numpy', 'SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_newton', 'zero')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_trust_region', 'zero')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                     $n=10$        \
Option                                           time prep/sol iters   
('numpy', 'Laplace', 'my_newton', 'zero')        0.002 / 0.018  15.0   
('numpy', 'Laplace', 'my_trust_region', 'zero')  0.002 / 0.086  14.0   
('numba', 'Laplace', 'my_newton', 'zero')        1.008 / 0.004  15.0   
('numba', 'Laplace', 'my_trust_region', 'zero')  1.008 / 0.049  14.0   
('jax', 'Laplace', 'my_newton', 'zero')          0.149 / 0.034  15.0   
('jax', 'Laplace', 'my_trust_region', 'zero')     0.149 / 0.11  14.0   
('numpy', 'SFD', 'my_newton', 'zero')            0.002 / 0.017   8.0   
('numpy', 'SFD', 'my_trust_region', 'zero')      0.002 / 0.056   9.0   
('numba', 'SFD', 'my_newton', 'zero')            1.068 / 0.005   8.0   
('numba', 'SFD', 'my_trust_region', 'zero')      1.068 / 0.029   9.0   
('jax', 'SFD', 'my_newton', 'zero')              0.146 / 0.027   9.0   
('jax', 'SFD', 'my_trust_region', 'zero')        0.146 / 0.073   9.0   
('jax', 'sparse', 'my_newton', 'zero')           0.372 / 0.099  42.0   
('jax', 'sparse', 'my_trust_region', 'zero')     0.372 / 0.088   9.0   

$\mathcal{N}_{dof}$                                    $n=100$          \
Option                                           time prep/sol   iters   
('numpy', 'Laplace', 'my_newton', 'zero')        0.001 / 0.043    37.0   
('numpy', 'Laplace', 'my_trust_region', 'zero')    0.001 / 0.3    43.0   
('numba', 'Laplace', 'my_newton', 'zero')        0.816 / 0.007    37.0   
('numba', 'Laplace', 'my_trust_region', 'zero')  0.816 / 0.173    43.0   
('jax', 'Laplace', 'my_newton', 'zero')          0.149 / 0.135    37.0   
('jax', 'Laplace', 'my_trust_region', 'zero')    0.149 / 0.517    43.0   
('numpy', 'SFD', 'my_newton', 'zero')            0.005 / 0.015     8.0   
('numpy', 'SFD', 'my_trust_region', 'zero')      0.005 / 0.133    18.0   
('numba', 'SFD', 'my_newton', 'zero')            0.987 / 0.011     8.0   
('numba', 'SFD', 'my_trust_region', 'zero')      0.987 / 0.063    18.0   
('jax', 'SFD', 'my_newton', 'zero')              0.162 / 0.035     8.0   
('jax', 'SFD', 'my_trust_region', 'zero')        0.162 / 0.217    18.0   
('jax', 'sparse', 'my_newton', 'zero')           0.451 / 1.389  1000.0   
('jax', 'sparse', 'my_trust_region', 'zero')     0.451 / 0.538    47.0   

$\mathcal{N}_{dof}$                                   $n=1000$          
Option                                           time prep/sol   iters  
('numpy', 'Laplace', 'my_newton', 'zero')        0.002 / 0.171    63.0  
('numpy', 'Laplace', 'my_trust_region', 'zero')  0.002 / 0.786    73.0  
('numba', 'Laplace', 'my_newton', 'zero')        0.921 / 0.048    63.0  
('numba', 'Laplace', 'my_trust_region', 'zero')  0.921 / 0.445    73.0  
('jax', 'Laplace', 'my_newton', 'zero')            0.26 / 0.45    63.0  
('jax', 'Laplace', 'my_trust_region', 'zero')     0.26 / 1.184    73.0  
('numpy', 'SFD', 'my_newton', 'zero')            0.062 / 0.047    13.0  
('numpy', 'SFD', 'my_trust_region', 'zero')      0.062 / 0.336    27.0  
('numba', 'SFD', 'my_newton', 'zero')              0.9 / 0.028    13.0  
('numba', 'SFD', 'my_trust_region', 'zero')        0.9 / 0.148    25.0  
('jax', 'SFD', 'my_newton', 'zero')              0.287 / 0.129    14.0  
('jax', 'SFD', 'my_trust_region', 'zero')        0.287 / 0.501    28.0  
('jax', 'sparse', 'my_newton', 'zero')             0.44 / 2.37  1000.0  
('jax', 'sparse', 'my_trust_region', 'zero')     0.44 / 10.153   485.0

# Initial guess as solution of Laplace (p=2)
Exact hessian (in jax autodiff) very much strougle when starting from 0. It is in orders of magnitude faster when starting from initial guess as solution of Laplace (p=2) (which is cheap in comparison).

In [11]:
all_solvers = []
all_solvers.append({"val_grad": "numpy",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "Laplace",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numpy",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "numba",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "SFD",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
all_solvers.append({"val_grad": "jax",
                    "hess": "sparse",
                    "minimizer": ["my_newton", "my_trust_region"],
                    "initial_guess": "laplace",
                    "problem_setting": problem_setting,
                    "sizes": sizes})
results = test_runner_1D.test_runner_1D(all_solvers)
iterations_df, f_val_df, compile_df, solve_df, combined_time_df, combined_df, combined_df2 = test_runner_1D.create_tables(
    results, display_table="Ipython")


Function value:


,10,100,1000
"('numpy', 'Laplace', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numpy', 'Laplace', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'Laplace', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('jax', 'Laplace', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547
"('numpy', 'SFD', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numpy', 'SFD', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_newton', 'laplace')",-16.772467,-16.864466,-16.86547
"('numba', 'SFD', 'my_trust_region', 'laplace')",-16.772467,-16.864466,-16.86547


Time and iters:


$\mathcal{N}_{dof}$                                        $n=10$        \
Option                                              time prep/sol iters   
('numpy', 'Laplace', 'my_newton', 'laplace')        0.003 / 0.008   8.0   
('numpy', 'Laplace', 'my_trust_region', 'laplace')  0.003 / 0.068  14.0   
('numba', 'Laplace', 'my_newton', 'laplace')         0.93 / 0.001   8.0   
('numba', 'Laplace', 'my_trust_region', 'laplace')   0.93 / 0.056  14.0   
('jax', 'Laplace', 'my_newton', 'laplace')           0.16 / 0.015   8.0   
('jax', 'Laplace', 'my_trust_region', 'laplace')     0.16 / 0.109  14.0   
('numpy', 'SFD', 'my_newton', 'laplace')            0.002 / 0.004   4.0   
('numpy', 'SFD', 'my_trust_region', 'laplace')      0.002 / 0.024   7.0   
('numba', 'SFD', 'my_newton', 'laplace')            0.761 / 0.003   4.0   
('numba', 'SFD', 'my_trust_region', 'laplace')      0.761 / 0.016   7.0   
('jax', 'SFD', 'my_newton', 'laplace')              0.144 / 0.012   4.0   
('jax', 'SFD', 'my_trust_region', 'laplace')        0.144 / 0.049   7.0   
('jax', 'sparse', 'my_newton', 'laplace')            0.33 / 0.008   4.0   
('jax', 'sparse', 'my_trust_region', 'laplace')      0.33 / 0.036   7.0   

$\mathcal{N}_{dof}$                                       $n=100$        \
Option                                              time prep/sol iters   
('numpy', 'Laplace', 'my_newton', 'laplace')        0.001 / 0.048  35.0   
('numpy', 'Laplace', 'my_trust_region', 'laplace')  0.001 / 0.172  30.0   
('numba', 'Laplace', 'my_newton', 'laplace')         1.148 / 0.01  35.0   
('numba', 'Laplace', 'my_trust_region', 'laplace')  1.148 / 0.113  30.0   
('jax', 'Laplace', 'my_newton', 'laplace')          0.139 / 0.111  35.0   
('jax', 'Laplace', 'my_trust_region', 'laplace')    0.139 / 0.367  30.0   
('numpy', 'SFD', 'my_newton', 'laplace')            0.005 / 0.005   5.0   
('numpy', 'SFD', 'my_trust_region', 'laplace')      0.005 / 0.049  10.0   
('numba', 'SFD', 'my_newton', 'laplace')             0.75 / 0.004   5.0   
('numba', 'SFD', 'my_trust_region', 'laplace')       0.75 / 0.027  10.0   
('jax', 'SFD', 'my_newton', 'laplace')              0.173 / 0.022   5.0   
('jax', 'SFD', 'my_trust_region', 'laplace')         0.173 / 0.08  10.0   
('jax', 'sparse', 'my_newton', 'laplace')           0.349 / 0.014   5.0   
('jax', 'sparse', 'my_trust_region', 'laplace')     0.349 / 0.081  10.0   

$\mathcal{N}_{dof}$                                      $n=1000$        
Option                                              time prep/sol iters  
('numpy', 'Laplace', 'my_newton', 'laplace')        0.004 / 0.176  61.0  
('numpy', 'Laplace', 'my_trust_region', 'laplace')   0.004 / 0.67  66.0  
('numba', 'Laplace', 'my_newton', 'laplace')         0.881 / 0.04  61.0  
('numba', 'Laplace', 'my_trust_region', 'laplace')  0.881 / 0.351  66.0  
('jax', 'Laplace', 'my_newton', 'laplace')          0.166 / 0.349  61.0  
('jax', 'Laplace', 'my_trust_region', 'laplace')    0.166 / 0.877  66.0  
('numpy', 'SFD', 'my_newton', 'laplace')            0.078 / 0.032   8.0  
('numpy', 'SFD', 'my_trust_region', 'laplace')      0.078 / 0.087  11.0  
('numba', 'SFD', 'my_newton', 'laplace')            1.104 / 0.015   8.0  
('numba', 'SFD', 'my_trust_region', 'laplace')      1.104 / 0.056  11.0  
('jax', 'SFD', 'my_newton', 'laplace')               0.23 / 0.046   8.0  
('jax', 'SFD', 'my_trust_region', 'laplace')         0.23 / 0.122  11.0  
('jax', 'sparse', 'my_newton', 'laplace')           0.464 / 0.022   6.0  
('jax', 'sparse', 'my_trust_region', 'laplace')     0.464 / 0.111  10.0